In [1]:
import os

using_colab = False
if using_colab:
  os.system('rm -rf ./*')
  os.system('git clone https://github.com/gasparyanartur/radio-optimizer.git')
  os.system('mv radio-optimizer/* ./')
  os.system('rm -rf radio-optimizer')

In [2]:
import numpy as np
from src.channel_mmWave_parameters import ChannelmmWaveParameters

seed = 0
rng = np.random.default_rng(seed)

In [1]:
import numpy as np
from src.utils import db2pow

from src.channel_mmWave_parameters import ChannelmmWaveParameters

c = ChannelmmWaveParameters(
    OB = np.array([45, 0, 0]).reshape(-1, 1),
    NB_dim = np.array([1, 1]).reshape(-1, 1),
    PR = np.array([[-5.01, 2, 0.5], [-1, 5.01, 0.5], [5.01, 2.5, 0.5]]).T,
    OR = np.array([[0, 0, 0], [-90, 0, 0], [180, 0, 0]]).T,
    NR_dim = np.array([[10, 10], [10, 10], [10, 10]]).T,
    PU = np.array([-2, 3, -1.5]).reshape(-1, 1),
    OU = np.array([0, 0, 0]).reshape(-1, 1),
    NU_dim = np.array([1, 1]).reshape(-1, 1),
    P = db2pow(15),
    K = 64,
    G = 10,

    seed=0
)


print(c.__dict__)

{'seed': 0, 'syn_type': <SynType.Asyn: 2>, 'link_type': <LinkType.Uplink: 1>, 'beam_type': <BeamType.Random: 1>, 'ris_profile_type': <RisProfileType.Random: 1>, 'beamforming_angle_std': 0, 'array_type': <ArrayType.Analog: 1>, 'radio_directional': 1, 'UE_radiation': <RadiationType.Omni: 1>, 'RIS_radiation': <RadiationType.Omni: 1>, 'BS_radiation': <RadiationType.Omni: 1>, 'c': 300000000.0, 'fc': 28000000000.0, 'lambdac': 0.010714285714285714, 'BW': 400000000.0, 'P': 31.622776601683793, 'beta': 30, 'K': 64, 'T': 0.001, 'PB': array([[0],
       [0],
       [0]]), 'OB': array([[45],
       [ 0],
       [ 0]]), 'PU': array([[-2. ],
       [ 3. ],
       [-1.5]]), 'OU': array([[0],
       [0],
       [0]]), 'VU': array([[0],
       [0],
       [0]]), 'PR': array([[-5.01, -1.  ,  5.01],
       [ 2.  ,  5.01,  2.5 ],
       [ 0.5 ,  0.5 ,  0.5 ]]), 'OR': array([[  0, -90, 180],
       [  0,   0,   0],
       [  0,   0,   0]]), 'VR': array([[0],
       [0],
       [0]]), 'MB': 1, 'MU': 1, 'NB_d

In [3]:
np.exp(2*np.pi*1j)

(1-2.4492935982947064e-16j)

In [4]:
from src.channel_mmWave_parameters import get_array_layout

get_array_layout(np.array([[10], [10]]))

array([[ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ],
       [-4.5, -4.5, -4.5, -4.5, -4.5, -4.5, -4.5, -4.5, -4.5, -4.5, -3.5,
        -3.5, -3.5, -3.5, -3.5, -3.5, -3.5, -3.5, -3.5, -3.5, -2.5, -2.5,
        -2.5, -2.5, -2.5, -2.5, -2.5, -2.5, -2.5, -2.5, -1.5, -1.5, -1.5,
        -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -0.5, -0.5, -0.5, -0.5,
        -0.5, -0.5, -0.

In [5]:
np.arange(5,2)*5 

array([], dtype=int32)

In [6]:
from scipy.spatial.transform import Rotation

Rotation.from_euler('ZYX', np.array([90, 45, 0]).reshape(1, -1).flatten(), degrees=True).as_matrix().shape


(3, 3)

In [7]:
len(np.arange(10).reshape(1, 5, 2).shape)

3

In [8]:
np.arange(6).reshape(2, 3).T

array([[0, 3],
       [1, 4],
       [2, 5]])

In [9]:
import numpy as np
from src.channel_mmWave_parameters import get_array_layout

get_array_layout(np.array([4, 4]).reshape(-1, 1))

array([[ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ],
       [-1.5, -1.5, -1.5, -1.5, -0.5, -0.5, -0.5, -0.5,  0.5,  0.5,  0.5,
         0.5,  1.5,  1.5,  1.5,  1.5],
       [-1.5, -0.5,  0.5,  1.5, -1.5, -0.5,  0.5,  1.5, -1.5, -0.5,  0.5,
         1.5, -1.5, -0.5,  0.5,  1.5]])

In [10]:
"""
"""
import numpy as np

def get_array_layout(N_dim: np.ndarray) -> np.ndarray:
    r, c = N_dim
    X, Y = np.meshgrid(np.arange(c) - (c-1)/2, np.arange(r) - (r-1)/2)

    XF = X.ravel(order="F")
    YF = Y.ravel(order="F")
    Z = np.zeros(r*c)

    layout = np.vstack((Z, XF, YF))
    return layout


get_array_layout((2, 4))

array([[ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [-1.5, -1.5, -0.5, -0.5,  0.5,  0.5,  1.5,  1.5],
       [-0.5,  0.5, -0.5,  0.5, -0.5,  0.5, -0.5,  0.5]])

In [11]:
rm = np.arange(9).reshape(3, 3)
v = np.array([4, 1, 2])

print(rm, v)
print(rm*v)
print(rm@v)

[[0 1 2]
 [3 4 5]
 [6 7 8]] [4 1 2]
[[ 0  1  4]
 [12  4 10]
 [24  7 16]]
[ 5 26 47]


In [13]:
np.linalg.norm(c.PB-c.PR, axis=0)

array([5.41757326, 5.13323485, 5.62139662])